$\renewcommand{\r}{\mathbb{ R}}$ $\newcommand{\Q}{\mathbb{ Q}}$ $\newcommand{\R}{\mathbb{ R}}$ $\newcommand{\C}{\mathbb{C}}$ $\newcommand{\Z}{\mathbb{Z}}$ $\newcommand{\ens}[1]{\left\{ {#1} \right\}}$ $\newcommand{\pv}{\;;\;}$ $\newcommand{\sev}{sous-espace vectoriel}$ $\newcommand{\evc}{espace vectoriel canonique}$ $\newcommand{\vect}{\text{vect}}$$\newcommand{\calf}{\mathscr{F}}$ $\newcommand{\calc}{\mathscr{C}}$



# TP n°4 : l'algorithme du pivot de Gauss vertical

*Vous devez disposer du cours d'Algèbre linéaire où l'algorithme du pivot vertival est expliqué. D'ailleurs, l'exemple utilisé dans le cours sera repris ici, donc ça vous permettra facilement de déboguer.*

**Si vous réussissez à coder honnêtement en 2 ou 3 heures à coder l'algorithme du pivot de Gauss sans mémorisation, vous avez alors obtenu un résultat plus qu'honorable**

On se donne un $K$-espace vectoriel $E$ de dimension $n$, muni d'une base $\calc=(c_0,\dots, c_{n-1})$. On se donne une famille $\calf=(u_0,u_0, \dots, u_{p-1})$ de $p$ vecteurs de $E$ dont les coordonnées dans la base $\calc$ sont connues. Plus précisément, soit la matrice $P$ de taille $n\times p$ dont les $p$ vecteurs-colonnes sont les coordonnées de $u_0,u_0, \dots, u_{p-1}$ dans la base $\calc$.  On notera $F=\vect(u_0,u_0, \dots, u_{p-1})$.

Dans tout l'exercice, l'exemple-témoin sera obtenu pour $n=5$, $p=4$ et la matrice $P$ suivante :

$$P=\left(\begin{array}{cccc} 0 & 1 & 2 &1\\ 0 & 0 &0 &0\\-2 & 1 & -1 &0\\0 & -2 &3&5\\ 1 &0 & 1& 0\end{array}\right)$$

correspondant aux vecteurs suivants :

$$\begin{array}{llll}u_0=( 0, 0, -2 , 0 , 1 )\\u_1=( 1, 0, 1 , -2 , 0 )\\u_2=( 2, 0, -1 , 3 , 1 )\\u_3=( 1, 0, 0 , 5 , 0)\end{array}$$

Toutes les indexations se font à partir de 0. De préférence, la matrice initiale $P$ devra être préservée (donc l'algorithme travaillera sur une copie $G$ de $P$).

Les matrices seront construites à coefficients dans $\Q$(observer `QQ` dans le code ci-dessous).


# Vecteurs et matrices : résumé

Vous êtes censés avoir lu la feuille sur l'utilisation rudimentaire des vecteurs et matrices sous SageMath. Voici toutefois quelques rappels sur les matrices dans une cellule de code ci-dessous.

In [ ]:
u1=vector([0, 0, -2 , 0 , 1])
u2=vector([1, 0, 1 , -2 , 0])
u3=vector([2, 0, -1 , 3 , 1])
u4=vector([1, 0, 0 , 5 , 0])

P=matrix(QQ,[u1, u2, u3,u4]).transpose()
show(P)

Pour faire une copie G de P, on utilise la fonction copy :

In [ ]:
G=copy(P)
show(G)

# on modifie la copie
G[0,0]=9999
show(G)

# on vérifie que P est préservée
show(P)

In [3]:
# Rappel des commandes utiles de calcul matriciel


# Une matrice à coeff. dans Q
# On écrit les lignes
P=matrix(QQ, [(33, 21, 12),(19, 63, 42),(81, 75, 20)])
show(P)

# nombres de lignes et de colonnes
n=P.nrows()
p=P.ncols()
print(n, p)

# lecture d'un élément
print(P[2,0])

# Modification d'un élément
P[2,0]=1000
show(P)

# La transposée
# permet d'écrire en colonne

PP=P.transpose()
show(PP)

# les lignes d'une matrices sont des vecteurs
print(type(P[2]))


# on peut aussi écrire directement les colonnes de la matrice

u1=vector([0, 0, -2 , 0 , 1])
u2=vector([1, 0, 1 , -2 , 0])
u3=vector([2, 0, -1 , 3 , 1])
u4=vector([1, 0, 0 , 5 , 0])

P=column_matrix(QQ,[u1, u2, u3,u4])
show(P)


[33 21 12]
[19 63 42]
[81 75 20]

3 3
81


[  33   21   12]
[  19   63   42]
[1000   75   20]

[  33   19 1000]
[  21   63   75]
[  12   42   20]

<class 'sage.modules.vector_rational_dense.Vector_rational_dense'>


[ 0  1  2  1]
[ 0  0  0  0]
[-2  1 -1  0]
[ 0 -2  3  5]
[ 1  0  1  0]

## Partie I : pivot de Gauss vertical simple

### Recherche de la position du pivot

On cherche la position du pivot dans la matrice $G$ **à partir** d'une certaine colonne d'indice $d$. Au début de l'algorithme, $d=0$. Il est à la ligne d'indice $i$ minimal et à la colonne d'indice $j\geq d$ minimal en sorte que le terme en position $(i,j)$ de $G$ soit non nul.

Ecrire une fonction `pos_pivot(G, col_depart)` qui renvoie les deux indices sous forme de liste `[i, j]`. Si tous les termes concernés dans la matrice `G` sont nuls, la fonction renverra la liste `[n+1, d]`.

On testera avec toutes les matrices de l'exemples ci-dessous.



In [ ]:
tests=[
      matrix([(4, 2),(8, 1)]),
      matrix([(0, 0, 4, 2),(0, 0, 0, 0),(0, 0, 0, 0)]),
      matrix([(0, 0, 0, 0),(0, 0, 4, 0),(0, 0, 0, 0)]),
      matrix([(0, 0, 0, 0),(2, 0, 0, 0),(0, 0, 0, 4)]),
      matrix([(0, 0, 0, 0),(0, 0, 0, 0),(0, 0, 0, 4)]),
      matrix([(0, 0, 0, 0),(0, 0, 0, 0),(0, 0, 0, 0)])]
show(tests)

In [ ]:
# Votre code ICI

# Tests
for P in tests:
    print(indice_col_non_nulle(P, 0))

### Échange de colonnes

L'algorithme du pivot vertical peut nécessiter d'échanger des colonnes. 

Le code ci-dessous montre, sur deux exemples, la manière pythonique d'échanger deux valeurs numériques (ici 42 et 81) :



In [ ]:
# 1er exemple
a=42
b=81
print( a,b)
# echange
a, b = b, a
print(a,b)
matrix([(33, 21, 12),(19, 63, 42),(81, 75, 20)])

Mais, avec la même syntaxe, on peut échanger des coefficients d'une matrice. Observez le code ci-dessous :

In [ ]:
# 2e exemple
P=matrix([(33, 21, 12),(19, 63, 42),(81, 75, 20)])
show(P)
# echange
P[2,0], P[1,2] = P[1,2],P[2,0]

show(P)

Ecrire une fonction `echanger_cols(G, j, k)` qui échange les colonnes d'indices `j` et `k` de `G`. On échangera élément par élément. La fonction ne renverra rien, elle se contentera de modifier `G`.

Bien sûr, il faut boucler sur le nombre de lignes.

Tester.



In [ ]:
# Votre code ICI

# Tests
        
P=matrix(QQ, [(33, 21, 12),(2, 64, 42),(81, 75, 20),(34, 49, 28)])
show(P)
echange_cols(P,0,2)
show(P)

### Réduction de colonne

On suppose que le pivot est à sa place, à la position `(ipivot, jpivot)`. Et on veut réduire la colonne d'indice `k` de `G` par la méthode du pivot de Gauss en colonnes. On rappelle qu'on ajoute à la colonne `Ck` d'indice `k`, la colonne `C` obtenue en multipliant la colonne d'indice `jpivot` par un coefficient adéquat pour que le terme en position `(ipivot, k)` devienne nul. On rappelle que SageMath travaille par défaut avec des fractions.

Coder une fonction `reduire_colk(G, ipivot, jpivot, k)` qui **modifie** `G` comme indiqué ci-dessus.

La fonction ne renvoie rien, elle a juste un effet de bord consistant à modifier un argument qu'elle reçoit.

Tester abondamment, sur des matrices arbitraires (pas forcément échelonnées, il suffit de se fixer un pivot non nul).



In [ ]:
# Votre code ICI


       
# exemple de test    
P=matrix(QQ, [(33, 21, 12),(2, 64, 42),(81, 75, 20),(34, 49, 28)])
show(P)

ipivot, jpivot = 1, 0
k=1
reduire_colk(P,ipivot, jpivot, k)
show(P)        

### Réduction de la ligne

On suppose toujours que le pivot est à sa place, à la position (ipivot, jpivot). Et on veut maintenant réduire **toute** la ligne située strictement à droite du pivot (et pas simplement un seul coefficient comme c'était le cas ci-dessus).

Coder une fonction `reduire_ligne(G, ipivot, jpivot)` qui **modifie** `G` comme indiqué ci-dessus. Cette fonction appelera en boucle la fonction précédente de reduction de colonne.

Tester.



In [ ]:
# Votre code ICI


# exemple de test    
P=matrix(QQ, [(33, 21, 12),(2, 64, 42),(81, 75, 20),(34, 49, 28)])
G=copy(P)
show(G)

ipivot, jpivot = 1, 0
reduire_ligne(G, ipivot, jpivot)
show(G)
print('===================')

G=copy(P)
show(G)

ipivot, jpivot = 0, 0
reduire_ligne(G, ipivot, jpivot)
show(G)

### L'algorithme du pivot de Gauss vertical

Utiliser toutes les fonctions précédentes pour coder une fonction `gauss(P)` qui implémente l'algorithme du pivot de Gauss vertical. La fonction devra renvoyer la matrice échelonnée en colonnes.

Pour une meilleure lisibilité, replacer toutes les fonctions utiles dans la cellule ci-dessous.

Tester avec tous les exemples ci-dessous. 

Pour obtenir une copie de `P`, utiliser la fonction `copy`.



In [ ]:

# Votre code ICI

# Tests   
u1=vector([0, 0, -2 , 0 , 1])
u2=vector([1, 0, 1 , -2 , 0])
u3=vector([2, 0, -1 , 3 , 1])
u4=vector([1, 0, 0 , 5 , 0])

P=column_matrix(QQ,[u1, u2, u3,u4])
E=gauss(P)
show(E)


P=matrix(QQ,[(33, 21, 12),(19, 63, 42),(81, 75, 20)])
E=gauss(P)
show(E)

P=matrix(QQ,[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])
E=gauss(P)
show(E)

P=matrix(QQ,4, 5)
E=gauss(P)
show(E)


P=matrix(QQ,[[1, 2, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])
show(P)
E=gauss(P)
show(E)

P=matrix(QQ, [(0, 5),(0, 2),(3, 4),(8, 9)])
show(P)
E=gauss(P)
show(E)

P=matrix(QQ, [(0, 5),(0, 2),(3, 4),(8, 9)]).transpose()
show(P)
E=gauss(P)
show(E)

### Image d'une application linéaire, rang

A l'aide de la fonction précédente, écrire une fonction `image(P)` qui renvoie une base de l'image de l'application linéaire $f$ canoniquement associée à `P` (attention, penser à retirer les vecteurs nuls des colonnes). Votre base sera juste une liste de vecteurs.

De même, écrire une fonction `rang(P)` qui renvoie le rang de $f$.



In [ ]:
# Votre code ICI

# Tests

u1=vector([0, 0, -2 , 0 , 1])
u2=vector([1, 0, 1 , -2 , 0])
u3=vector([2, 0, -1 , 3 , 1])
u4=vector([1, 0, 0 , 5 , 0])

P=column_matrix(QQ,[u1, u2, u3,u4])

print(image(P), rang(P))

## Algorithme de Gauss avec mémorisation

Le cours détaille le principe de la mémorisation. Le relire avant de vous lancer dans le codage, c'est moins simple que l'algorithme sans mémorisation. Les colonnes de chaque nouveau tableau dans un pivot de Gauss vertical sont une combinaison linéaire des $p$ vecteurs initiaux. L'algorithme de pivot vertical avec mémorisation consiste à effectuer le pivot de Gauss vertical mais à garder trace, à chaque nouveau tableau, de cette combinaison linéaire. Pour garder trace de chaque combinaison linéaire, il suffit de pronlonger chaque colonne avec $p$ lignes dont les $p$ coefficients représentent les coefficients de la combinaison linéaire donnant la colonne référencée. Ainsi, la matrice $P$ suivante :

$$P=\left(\begin{array}{cccc} 0 & 1 & 2 &1\\ 0 & 0 &0 &0\\-2 & 1 & -1 &0\\0 & -2 &3&5\\ 1 &0 & 1& 0\end{array}\right)$$

correspondant aux vecteurs suivants :

$$\begin{array}{llll}u_0=( 0, 0, -2 , 0 , 1 )\\u_1=( 1, 0, 1 , -2 , 0 )\\u_2=( 2, 0, -1 , 3 , 1 )\\u_3=( 1, 0, 0 , 5 , 0)\end{array}$$

sera initialisée par le tableau prolongé suivant :

$$\begin{array}{cccc}0 & 1 & 2 &1\\ 0 & 0 &0 &0\\-2 & 1 & -1 &0\\0 & -2 &3&5\\1 &0 & 1& 0\\\hline 1 & 0 & 0 &0\\0 & 1 &0 &0\\0 & 0 & 1 &0\\0 & 0 &0&1\end{array}$$

Noter qu'à la différence du cours j'ai placé la mémorisation en-dessous mais cela n'a aucune conséquence.

L'intérêt de cette présentation est qu'il suffit de faire le pivot de Gauss vertical habituel sur la matrice prolongée pour garder des traces des valeurs des colonnes (ce point est essentiel à comprendre, regarder les exemples du cours si ce n'est pas clair.

Utiliser la fonction `gauss(P)` de la question précédente dans une fonction `gauss_memo(P)` qui renvoie une liste `[E, T]` de deux matrices :

- la matrice échelonnée `E` de taille `n` x `p` (celle de l'algo sans mémorisation) ;

- la matrice `T` carrée d'ordre `p` dont les lignes sont les coefficients des combinaisons linéaires des `p` vecteurs initiaux représentant les différentes colonnes de la matrice échelonnée `E`.

Tester et vérifier.



In [ ]:
# Votre code ICI

# Tests
        
u1=vector([0, 0, -2 , 0 , 1])
u2=vector([1, 0, 1 , -2 , 0])
u3=vector([2, 0, -1 , 3 , 1])
u4=vector([1, 0, 0 , 5 , 0])

P=column_matrix(QQ,[u1, u2, u3,u4])
show(P)
show(gauss_mem(P))

<html><script type="math/tex; mode=display">\newcommand{\Bold}[1]{\mathbf{#1}}\left(\begin{array}{rrrr}

0 ` 1 ` 2 & 1 \\

0 ` 0 ` 0 & 0 \\

- 2 ` 1 ` -1 & 0 \\

0 ` -2 ` 3 & 5 \\

1 ` 0 ` 1 & 0

\end{array}\right)</script></html>

<html><script type="math/tex; mode=display">\newcommand{\Bold}[1]{\mathbf{#1}}\left(\left(\begin{array}{rrrr}

1 ` 0 ` 0 & 0 \\

0 ` 0 ` 0 & 0 \\

1 ` -2 ` 0 & 0 \\

- 2 ` 0 ` 7 & 0 \\

0 ` 1 ` -\frac{1}{2} & 0

\end{array}\right), \left(\begin{array}{rrrr}

0 ` 1 ` 0 & 0 \\

1 ` 0 ` 0 & 0 \\

- \frac{3}{2} ` -2 ` 1 & 0 \\

1 ` 1 ` -1 & 1

\end{array}\right)\right)</script></html>

## Partie III : équations cartésiennes

L'algorithme du pivot vertical permet de déterminer un système d'équations cartésiennes du sev engendré par les vecteurs de la famille $(u_0,u_0, \dots, u_{p-1})$. Pour cela, il suffit d'adjoindre au tableau une colonne symbolique $x_0,\dots, x_{n-1}$

$$\begin{array}{cccc|c} 0 & 1 & 2 &1&x_0\\ 0 & 0 &0 &0&x_1\\-2 & 1 & -1 &0&x_2\\0 & -2 &3&5&x_3\\ 1 &0 & 1& 0&x_4\end{array}$$

et d'effectuer l'algorithme habituel sauf que les expressions symboliques ne deviennent jamais pivot. En pratique, comme les coefficients des variables $x_i$ ne se modifient que par combinaison linéaire, on n'utilise pas une seule colonne symbolique mais plutôt $n$ colonnes chacune représentant le coefficient de $x_i$ :

$$\begin{array}{cccc|ccccc} 0 & 1 & 2 &1&1 & 0 & 0& 0& 0 \\ 0 & 0 &0 &0&0 & 1 & 0& 0& 0 \\-2 & 1 & -1 &0&0 & 0 & 1& 0& 0 \\0 & -2 &3&5&0 & 0 & 0& 1& 0 \\ 1 &0 & 1& 0&0 & 0 & 0& 0& 1 \\\end{array}$$

Modifier la fonction `gauss(P)` de la première question et créer une fonction `equat_cart(P)` qui renvoie les équations cartésiennes du sev engendré $F$ par les vecteurs colonnes de `P`. Les équations cartésiennes seront données sous forme d'une matrice de $q$ lignes et $n$ colonnes, chaque ligne représentant les coefficients d'une des équations cartésiennes de $F$. On commencera par étendre vers la droite la matrice initiale. Pour adapter le code de `gauss`, il suffit en fait d'interdire la venue de pivots originaires d'une colonne d'indice $j>p-1$. Les équations cartésiennes seront obtenues en sélectionnant les lignes non nulles parmi les $n$ dernières colonnes de la matrice étendue.

Pour l'exemple témoin, on obtiendra la matrice suivante :

$$\left(\begin{array}{rrrrrrrrr}1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\1 & -2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\-2 & 0 & 7 & 0 & 0 & 0 & 0 & 0 & 0 \\0 & 1 & -\frac{1}{2} & 0 & -\frac{5}{14} & 0 & \frac{1}{2} & \frac{1}{14} & 1\end{array}\right)$$

ce qui signifie qu'un système d'équations cartésiennes de $F$ est

$$\begin{cases} x_1=0\\ -\frac{5}{14}x_0 +  \frac{1}{2}x_2 + \frac{1}{14}x_3 + x_4=0\end{cases}$$

Tester et vérifier.



In [ ]:
# Votre code ICI

# Tests
        
u1=vector([0, 0, -2 , 0 , 1])
u2=vector([1, 0, 1 , -2 , 0])
u3=vector([2, 0, -1 , 3 , 1])
u4=vector([1, 0, 0 , 5 , 0])

P=column_matrix(QQ,[u1, u2, u3,u4])
show(P)
ec(P)